# Estimate translational and Coriolis velocities

In [1]:
%matplotlib notebook

First we define some parameters

In [2]:
import numpy as np
# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [3]:
radcols=['64ne', '64se', '64sw', '64nw', '50ne', '50se', '50sw', '50nw',
       '34ne', '34se', '34sw', '34nw']

In [4]:
import matplotlib.pyplot as plt

In [5]:
import pandas as pd

## Holland Parameter Evaluation

The input data comes from the bulletins and are saved in txt format during the step 1.

In [6]:
path='test/'

In [7]:
filename='step1.txt'

Next the range of Holland parameters is defined (see 'Global storm surge forecast and inundation modeling' report by Probst & Franchello)

In [8]:
npmin=2 # ??????

kmin=0  # low limit of parameter k (=xn-.5-> k=0-> x=0.5)
kmax=0.15 # upper limit for k (->xn=.65)  WHY?

dpmin=10.e2  # minimum value of  pressure drop P_central - P_env(=101kPa).
dpmax=200.e2   # maximum value of  pressure drop P_central - P_env(=101kPa).
rvmaxmin=10.e3  # default minimum value of Rmax[m] 

bmin=0.8 # minimum value of holland parameter b
#bmax=2.5
bmax=1.8  # maximum value of holland parameter b
b0=1.5  # initial estimation of holland parameter b

rmax0=20.e3  # intial estimation for radius of maximum wind [m] (20km)
maxR=500.e3  # maximum radius of TC [m] (500Km)



In [9]:
fk=0.92  # coefficient for going from 1m to 10m in velocities ????????????????????????

Reading the data into a pandas DataFrame 

In [10]:
inpdat = pd.read_csv(path+filename, delimiter='\t')

In [11]:
inpdat.head()

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,34nw,50ne,50se,50sw,50nw,64ne,64se,64sw,64nw,dp
0,0.0,2010-10-29 06:00:00,INVEST,9.0,-53.7,1006,1009,222240.0,15.433333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300
1,6.0,2010-10-29 12:00:00,INVEST,9.8,-55.3,1003,1009,166680.0,18.005556,138900.0,...,74080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600
2,12.0,2010-10-29 18:00:00,TOMAS,10.8,-56.8,998,1009,55560.0,23.150000,166680.0,...,92600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1100
3,18.0,2010-10-30 00:00:00,TOMAS,11.9,-57.8,999,1009,55560.0,28.294444,166680.0,...,92600.0,55560.0,0.0,55560.0,0.0,0.0,0.0,0.0,0.0,1000
4,24.0,2010-10-30 06:00:00,TOMAS,12.7,-58.9,997,1009,55560.0,30.866667,166680.0,...,92600.0,55560.0,0.0,55560.0,0.0,0.0,0.0,0.0,0.0,1200


Check if we cross the International Date Line

In [12]:
if inpdat.lon.apply(np.sign).diff().sum() > 0:
    m=inpdat.lon != inpdat.lon[0]
    inpdat.lon[m]+360. if inpdat.lon[0] > 0 else npdat.lon[m]-360.

## Calculate translation velocity

In [13]:
x=inpdat.lon
y=inpdat.lat

In [14]:
dt=np.gradient(inpdat.time)*3600 # compute dt (translate time from hours to sec)

In [15]:
dx_dt = np.gradient(x,dt)
dy_dt = np.gradient(y,dt)
velocity = np.array([ [dx_dt[i], dy_dt[i]] for i in range(dx_dt.size)])

In [16]:
#velocity

In [17]:
vtrx = velocity[:,0] * deg2m * np.cos(np.radians(inpdat.lat.values))  #adjust for latitude
vtry = velocity[:,1] * deg2m

In [18]:
vtr = np.sqrt(vtrx**2+vtry**2)

In [19]:
#print vtrx,vtry,vtr

Compute the tangent of unit vector value, see http://stackoverflow.com/questions/28269379/curve-curvature-in-numpy

In [20]:
ds_dt = np.sqrt(dx_dt * dx_dt + dy_dt * dy_dt)

In [21]:
tangent = np.array([1/ds_dt] * 2).transpose() * velocity

In [22]:
phi=np.arctan2(tangent[:,1],tangent[:,0]) # the angle of the velocity vector

In [23]:
cosfi = np.cos(phi)
sinfi = np.sin(phi)

In [24]:
# extend dataset to save new data
inpdat['vtrx']=vtrx
inpdat['vtry']=vtry
inpdat['vtr']=vtr
inpdat['cosfi']=cosfi
inpdat['sinfi']=sinfi

In [25]:
inpdat.head()

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,64ne,64se,64sw,64nw,dp,vtrx,vtry,vtr,cosfi,sinfi
0,0.0,2010-10-29 06:00:00,INVEST,9.0,-53.7,1006,1009,222240.0,15.433333,0.0,...,0.0,0.0,0.0,0.0,300,-8.144688,4.123106,9.128853,-0.894427,0.447214
1,6.0,2010-10-29 12:00:00,INVEST,9.8,-55.3,1003,1009,166680.0,18.005556,138900.0,...,0.0,0.0,0.0,0.0,600,-7.871949,4.638495,9.136915,-0.864789,0.502136
2,12.0,2010-10-29 18:00:00,TOMAS,10.8,-56.8,998,1009,55560.0,23.150000,166680.0,...,0.0,0.0,0.0,0.0,1100,-6.328242,5.411577,8.326573,-0.765705,0.643192
3,18.0,2010-10-30 00:00:00,TOMAS,11.9,-57.8,999,1009,55560.0,28.294444,166680.0,...,0.0,0.0,0.0,0.0,1000,-5.295277,4.896189,7.211978,-0.741536,0.670913
4,24.0,2010-10-30 06:00:00,TOMAS,12.7,-58.9,997,1009,55560.0,30.866667,166680.0,...,0.0,0.0,0.0,0.0,1200,-5.781959,3.092330,6.556947,-0.886585,0.462566


In [26]:
cols=['w'+ x for x in radcols]

In [27]:
#temp = np.zeros((time.size, 12))
#d = pd.DataFrame(temp, columns = cols)

In [28]:
inpdat

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,64ne,64se,64sw,64nw,dp,vtrx,vtry,vtr,cosfi,sinfi
0,0.0,2010-10-29 06:00:00,INVEST,9.0,-53.7,1006,1009,222240.0,15.433333,0.0,...,0.0,0.0,0.0,0.0,300,-8.144688,4.123106,9.128853,-0.894427,4.472136e-01
1,6.0,2010-10-29 12:00:00,INVEST,9.8,-55.3,1003,1009,166680.0,18.005556,138900.0,...,0.0,0.0,0.0,0.0,600,-7.871949,4.638495,9.136915,-0.864789,5.021355e-01
2,12.0,2010-10-29 18:00:00,TOMAS,10.8,-56.8,998,1009,55560.0,23.150000,166680.0,...,0.0,0.0,0.0,0.0,1100,-6.328242,5.411577,8.326573,-0.765705,6.431921e-01
3,18.0,2010-10-30 00:00:00,TOMAS,11.9,-57.8,999,1009,55560.0,28.294444,166680.0,...,0.0,0.0,0.0,0.0,1000,-5.295277,4.896189,7.211978,-0.741536,6.709133e-01
4,24.0,2010-10-30 06:00:00,TOMAS,12.7,-58.9,997,1009,55560.0,30.866667,166680.0,...,0.0,0.0,0.0,0.0,1200,-5.781959,3.092330,6.556947,-0.886585,4.625660e-01
5,30.0,2010-10-30 12:00:00,TOMAS,13.1,-60.1,993,1009,37040.0,33.438889,222240.0,...,37040.0,0.0,0.0,37040.0,1600,-5.270746,1.546165,5.492849,-0.961524,2.747211e-01
6,36.0,2010-10-30 18:00:00,TOMAS,13.3,-61.0,990,1009,37040.0,41.155556,222240.0,...,37040.0,0.0,0.0,37040.0,1900,-4.012520,1.030777,4.142803,-0.970143,2.425356e-01
7,42.0,2010-10-31 00:00:00,TOMAS,13.5,-61.7,982,1009,37040.0,43.727778,222240.0,...,55560.0,18520.0,37040.0,37040.0,2700,-3.508037,1.288471,3.737175,-0.941742,3.363364e-01
8,48.0,2010-10-31 06:00:00,TOMAS,13.8,-62.4,983,1009,27780.0,43.727778,222240.0,...,55560.0,18520.0,37040.0,37040.0,2600,-4.004090,1.288471,4.206292,-0.954480,2.982750e-01
9,54.0,2010-10-31 12:00:00,TOMAS,14.0,-63.3,988,1011,27780.0,41.155556,222240.0,...,55560.0,18520.0,37040.0,37040.0,2300,-5.000791,1.030777,5.105919,-0.980581,1.961161e-01


In [29]:
an=np.array([tetaNE, tetaSE, tetaSW, tetaNW,tetaNE, tetaSE, tetaSW, tetaNW,tetaNE, tetaSE, tetaSW, tetaNW])# to be used
sinan = np.sin(np.radians(an+90))  # an +90 = angle of tangential wind
cosan=np.cos(np.radians(an+90))

In [30]:
V0=np.array([64, 64, 64, 64, 50, 50, 50, 50, 34, 34, 34, 34])*kt2ms*fk #translate knots to m/s and from 1km to 10km

In [31]:
R=inpdat.ix[:,radcols].copy()

In [32]:
R=R[R>0]

In [33]:
RATIO = (rmax0/R)**b0    # assume exponential decay eqs (13) from JRC report
EXPRATIO = np.exp(-RATIO)  #                       "

In [34]:
RATIO

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.054638,NaN,0.054638,0.140279
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041564,0.100376,0.041564,0.100376
3,NaN,NaN,NaN,NaN,0.215974,NaN,0.215974,NaN,0.041564,0.100376,0.041564,0.100376
4,NaN,NaN,NaN,NaN,0.215974,NaN,0.215974,NaN,0.041564,0.100376,0.041564,0.100376
5,0.396770,NaN,NaN,0.396770,0.100376,0.396770,0.140279,0.215974,0.026997,0.100376,0.041564,0.076358
6,0.396770,NaN,NaN,0.396770,0.100376,0.396770,0.140279,0.215974,0.026997,0.100376,0.041564,0.076358
7,0.215974,1.122234,0.396770,0.396770,0.100376,0.396770,0.140279,0.215974,0.026997,0.100376,0.041564,0.076358
8,0.215974,1.122234,0.396770,0.396770,0.076358,0.396770,0.100376,0.215974,0.026997,0.100376,0.041564,0.076358
9,0.215974,1.122234,0.396770,0.396770,0.076358,0.396770,0.100376,0.215974,0.026997,0.100376,0.041564,0.076358


In [35]:
VT=vtr[:,np.newaxis]*(cosfi[:,np.newaxis] * cosan + sinfi[:,np.newaxis] * sinan)*(1-EXPRATIO)   # Eq (15) from JRC report

In [36]:
VT

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.695808e-01,NaN,-4.695808e-01,3.066653e-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.377194e-01,-0.068889,-3.377194e-01,6.888857e-02
3,NaN,NaN,NaN,NaN,1.399134e+00,NaN,-1.399134e+00,NaN,2.932497e-01,-0.034395,-2.932497e-01,3.439511e-02
4,NaN,NaN,NaN,NaN,1.215051e+00,NaN,-1.215051e+00,NaN,2.546670e-01,-0.187753,-2.546670e-01,1.877528e-01
5,1.572582,NaN,NaN,0.873657,4.585659e-01,-0.873657,-6.284570e-01,0.518158,1.278939e-01,-0.254759,-1.954847e-01,1.961083e-01
6,1.163459,NaN,NaN,0.698075,3.392652e-01,-0.698075,-4.649574e-01,0.414022,9.462097e-02,-0.203559,-1.446273e-01,1.566958e-01
7,0.656044,-1.079005,-1.106145,0.523963,3.225525e-01,-0.523963,-4.420529e-01,0.310758,8.995980e-02,-0.152788,-1.375027e-01,1.176132e-01
8,0.723765,-1.316354,-1.220328,0.639220,2.739250e-01,-0.639220,-3.558484e-01,0.379115,9.924603e-02,-0.186397,-1.516967e-01,1.434845e-01
9,0.825222,-1.910209,-1.391392,0.927595,3.123235e-01,-0.927595,-4.057308e-01,0.550148,1.131582e-01,-0.270487,-1.729613e-01,2.082156e-01


In [37]:
VT.loc[inpdat.lat<0] = -VT # reverse for south hemishpere

In [38]:
VT

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.695808e-01,NaN,-4.695808e-01,3.066653e-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.377194e-01,-0.068889,-3.377194e-01,6.888857e-02
3,NaN,NaN,NaN,NaN,1.399134e+00,NaN,-1.399134e+00,NaN,2.932497e-01,-0.034395,-2.932497e-01,3.439511e-02
4,NaN,NaN,NaN,NaN,1.215051e+00,NaN,-1.215051e+00,NaN,2.546670e-01,-0.187753,-2.546670e-01,1.877528e-01
5,1.572582,NaN,NaN,0.873657,4.585659e-01,-0.873657,-6.284570e-01,0.518158,1.278939e-01,-0.254759,-1.954847e-01,1.961083e-01
6,1.163459,NaN,NaN,0.698075,3.392652e-01,-0.698075,-4.649574e-01,0.414022,9.462097e-02,-0.203559,-1.446273e-01,1.566958e-01
7,0.656044,-1.079005,-1.106145,0.523963,3.225525e-01,-0.523963,-4.420529e-01,0.310758,8.995980e-02,-0.152788,-1.375027e-01,1.176132e-01
8,0.723765,-1.316354,-1.220328,0.639220,2.739250e-01,-0.639220,-3.558484e-01,0.379115,9.924603e-02,-0.186397,-1.516967e-01,1.434845e-01
9,0.825222,-1.910209,-1.391392,0.927595,3.123235e-01,-0.927595,-4.057308e-01,0.550148,1.131582e-01,-0.270487,-1.729613e-01,2.082156e-01


In [39]:
VV = V0-VT   # substract translational velocity from TC velocity

In [40]:
deltalatWR=R/deg2m*np.sin(np.radians(an))

In [41]:
deltalatWR

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.882265,NaN,-0.882265,0.470541
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.058718,-0.588177,-1.058718,0.588177
3,NaN,NaN,NaN,NaN,0.352906,NaN,-0.352906,NaN,1.058718,-0.588177,-1.058718,0.588177
4,NaN,NaN,NaN,NaN,0.352906,NaN,-0.352906,NaN,1.058718,-0.588177,-1.058718,0.588177
5,0.235271,NaN,NaN,0.235271,0.588177,-0.235271,-0.470541,0.352906,1.411624,-0.588177,-1.058718,0.705812
6,0.235271,NaN,NaN,0.235271,0.588177,-0.235271,-0.470541,0.352906,1.411624,-0.588177,-1.058718,0.705812
7,0.352906,-0.117635,-0.235271,0.235271,0.588177,-0.235271,-0.470541,0.352906,1.411624,-0.588177,-1.058718,0.705812
8,0.352906,-0.117635,-0.235271,0.235271,0.705812,-0.235271,-0.588177,0.352906,1.411624,-0.588177,-1.058718,0.705812
9,0.352906,-0.117635,-0.235271,0.235271,0.705812,-0.235271,-0.588177,0.352906,1.411624,-0.588177,-1.058718,0.705812


In [42]:
latWR=inpdat.lat[:,np.newaxis]+deltalatWR

In [43]:
latWR

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.682265,NaN,8.917735,10.270541
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.858718,10.211823,9.741282,11.388177
3,NaN,NaN,NaN,NaN,12.252906,NaN,11.547094,NaN,12.958718,11.311823,10.841282,12.488177
4,NaN,NaN,NaN,NaN,13.052906,NaN,12.347094,NaN,13.758718,12.111823,11.641282,13.288177
5,13.335271,NaN,NaN,13.335271,13.688177,12.864729,12.629459,13.452906,14.511624,12.511823,12.041282,13.805812
6,13.535271,NaN,NaN,13.535271,13.888177,13.064729,12.829459,13.652906,14.711624,12.711823,12.241282,14.005812
7,13.852906,13.382365,13.264729,13.735271,14.088177,13.264729,13.029459,13.852906,14.911624,12.911823,12.441282,14.205812
8,14.152906,13.682365,13.564729,14.035271,14.505812,13.564729,13.211823,14.152906,15.211624,13.211823,12.741282,14.505812
9,14.352906,13.882365,13.764729,14.235271,14.705812,13.764729,13.411823,14.352906,15.411624,13.411823,12.941282,14.705812


In [44]:
fWR=2*omega*np.abs(np.sin(np.radians(latWR))) # Coriolis parameter f=2*Omega*sin(lat)
Vnco=((VV+R*fWR/2)**2-(R*fWR/2)**2)**0.5

In [45]:
Vnco=Vnco.replace(np.nan,0)

In [46]:
Vnco

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.394121,0.000000,18.059481,16.718113
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.074184,17.313465,18.366306,17.301589
3,0.000000,0.000000,0.000000,0.000000,23.106896,0.000000,25.859722,0.000000,18.316167,17.397022,18.525193,17.452947
4,0.000000,0.000000,0.000000,0.000000,23.344100,0.000000,25.728973,0.000000,18.496702,17.636008,18.632523,17.383112
5,29.332609,0.000000,0.000000,30.031688,24.748161,25.130682,25.443546,24.067929,19.599667,17.745589,18.645319,17.719449
6,29.750778,0.000000,0.000000,30.216264,24.889111,24.963993,25.297372,24.185329,19.678071,17.715329,18.630080,17.783646
7,30.586552,31.679678,32.008721,30.399362,24.927226,24.798761,25.291917,24.301841,19.727123,17.685447,18.658875,17.847466
8,30.538677,31.923827,32.136380,30.297483,25.333873,24.927432,25.512994,24.253146,19.783799,17.750702,18.727010,17.858079
9,30.450398,32.522233,32.316436,30.017961,25.320538,25.224808,25.584497,24.095087,19.813501,17.856080,18.784244,17.817282


In [47]:
#change header
Vnco.columns = cols

In [48]:
# extend dataset to save the velocities
inpdat = pd.concat([inpdat, Vnco], axis=1)

In [49]:
inpdat.head()

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,w64sw,w64nw,w50ne,w50se,w50sw,w50nw,w34ne,w34se,w34sw,w34nw
0,0.0,2010-10-29 06:00:00,INVEST,9.0,-53.7,1006,1009,222240.0,15.433333,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,6.0,2010-10-29 12:00:00,INVEST,9.8,-55.3,1003,1009,166680.0,18.005556,138900.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,17.394121,0.000000,18.059481,16.718113
2,12.0,2010-10-29 18:00:00,TOMAS,10.8,-56.8,998,1009,55560.0,23.150000,166680.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,18.074184,17.313465,18.366306,17.301589
3,18.0,2010-10-30 00:00:00,TOMAS,11.9,-57.8,999,1009,55560.0,28.294444,166680.0,...,0.0,0.0,23.106896,0.0,25.859722,0.0,18.316167,17.397022,18.525193,17.452947
4,24.0,2010-10-30 06:00:00,TOMAS,12.7,-58.9,997,1009,55560.0,30.866667,166680.0,...,0.0,0.0,23.344100,0.0,25.728973,0.0,18.496702,17.636008,18.632523,17.383112


In [50]:
vs = inpdat.vmax*fk-vtr

In [51]:
vmax0vt = np.maximum(vs,Vnco.max(axis=1))

In [52]:
inpdat['vmax0vt'] = vmax0vt

In [53]:
inpdat = inpdat.set_index('time')


In [54]:
inpdat.head()

,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,34se,...,w64nw,w50ne,w50se,w50sw,w50nw,w34ne,w34se,w34sw,w34nw,vmax0vt
time,,,,,,,,,,,,,,,,,,,,,
0.0,2010-10-29 06:00:00,INVEST,9.0,-53.7,1006,1009,222240.0,15.433333,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,5.069814
6.0,2010-10-29 12:00:00,INVEST,9.8,-55.3,1003,1009,166680.0,18.005556,138900.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,17.394121,0.000000,18.059481,16.718113,18.059481
12.0,2010-10-29 18:00:00,TOMAS,10.8,-56.8,998,1009,55560.0,23.150000,166680.0,92600.0,...,0.0,0.000000,0.0,0.000000,0.0,18.074184,17.313465,18.366306,17.301589,18.366306
18.0,2010-10-30 00:00:00,TOMAS,11.9,-57.8,999,1009,55560.0,28.294444,166680.0,92600.0,...,0.0,23.106896,0.0,25.859722,0.0,18.316167,17.397022,18.525193,17.452947,25.859722
24.0,2010-10-30 06:00:00,TOMAS,12.7,-58.9,997,1009,55560.0,30.866667,166680.0,92600.0,...,0.0,23.344100,0.0,25.728973,0.0,18.496702,17.636008,18.632523,17.383112,25.728973


## save output to file

In [56]:
inpdat.to_csv(path+'step2.txt',index=True, sep='\t')

Now we estimating the Rmax. A number of ways apply. See [Estimate Holland Parameters.ipynb](./Estimate Holland Parameters.ipynb)